<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task1/blob/main/DL_RoBERTa_Task1_en_23_8_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 32.3 MB/s eta 0:00:00


In [2]:
#Leer los archivos TSV y seleccionar las columnas deseadas y filtrar por idioma 'es'

import pandas as pd
from sklearn.utils import shuffle
test_data = pd.read_csv('/content/EXIST2021_test_labeled.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'en']
train_data = pd.read_csv('/content/EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'en']
#train_data_en = pd.read_csv('/content/train_data_translated_en.csv', sep=',', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'en']
swapp = pd.read_csv('/content/swapp_EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'en']

In [3]:
# Concatenar train_data y train_data_en en un único DataFrame llamado train_data
train_data = pd.concat([train_data, swapp], ignore_index=True)
train_data = shuffle(train_data, random_state=42)

# Verificar el resultado
print(train_data.head())

     language                                               text       task1
6211       en  if a woman is telling you to spank her she doe...      sexist
8367       en   No fr why do us women hit curbs then laugh lmaoo      sexist
5456       en  @CoprinusGarden nope. i simply stated that tra...  non-sexist
4250       en  @harrz99 Imagine a conversation with that bint...  non-sexist
7758       en  TRIGGERED " by a TRANSIT PINK TAX That ' doesn...      sexist


In [4]:
columna_clase_objetivo = "task1"

df = train_data

# Contar la cantidad de categorías únicas en la columna de la clase objetivo
cantidad_categorias = df[columna_clase_objetivo].nunique()

# Contar la cantidad de muestras para cada categoría de la clase objetivo
muestras_por_categoria = df[columna_clase_objetivo].value_counts()

# Imprimir los resultados
print("Cantidad de categorías:", cantidad_categorias)
print("Muestras por categoría:")
print(muestras_por_categoria)
df.shape[0]

Cantidad de categorías: 2
Muestras por categoría:
non-sexist    4904
sexist        4468
Name: task1, dtype: int64


9372

#**PREPROCESADO**


---



In [5]:
import re

def anonimize_text(text):
    # Anonimizar nombres de usuario
    text = re.sub(r'@[\w_]+', '@user', text)

    # Anonimizar hashtags
    text = re.sub(r'#\w+', '#hashtag', text)

    # Anonimizar enlaces
    text = re.sub(r'https?://\S+', 'link', text)

    return text

train_data['text'] = train_data['text'].apply(anonimize_text)
test_data['text'] = test_data['text'].apply(anonimize_text)

In [6]:
# Convertir a minúsculas

train_data.text = train_data.text.str.lower()
train_data.head(10)
test_data.text = test_data.text.str.lower()
test_data.head(10)

,language,text,task1
0,en,pennsylvania state rep horrifies with opening ...,non-sexist
1,en,"@user he sounds like as ass, and very condesce...",non-sexist
2,en,"@user @user lol! ""this behavior of not letting...",sexist
3,en,@user @user rights?i mean yeah most women espe...,sexist
4,en,the jack manifold appreciation i’m seeing is o...,non-sexist
5,en,@user why isn't this character taking a lie de...,non-sexist
6,en,@user i can’t help but torture myself reading ...,non-sexist
7,en,@user @user @user stop regarding women as anim...,sexist
8,en,"yeah, it is rough, but not for women. married ...",sexist
9,en,although the christian denial isn't emphasized...,non-sexist


In [7]:
# Eliminar caráctres especiales, #:)(/\='] usados en emojis y hashtags

train_data.text = train_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
train_data.head(10)
test_data.text = test_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
test_data.head(10)

<ipython-input-7-091c5808df50>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data.text = train_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
<ipython-input-7-091c5808df50>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data.text = test_data.text.str.replace("[^a-zA-ZñÑ#]", " ")


,language,text,task1
0,en,pennsylvania state rep horrifies with opening ...,non-sexist
1,en,user he sounds like as ass and very condesce...,non-sexist
2,en,user user lol this behavior of not letting...,sexist
3,en,user user rights i mean yeah most women espe...,sexist
4,en,the jack manifold appreciation i m seeing is o...,non-sexist
5,en,user why isn t this character taking a lie de...,non-sexist
6,en,user i can t help but torture myself reading ...,non-sexist
7,en,user user user stop regarding women as anim...,sexist
8,en,yeah it is rough but not for women married ...,sexist
9,en,although the christian denial isn t emphasized...,non-sexist




---



In [8]:
# Preprocess the dataset
train_data['task1'] = train_data['task1'].map({'non-sexist': 1, 'sexist': 0}) # Convert labels to integers
train_data['text'] = train_data['text'].astype(str) # Ensure all data are strings

In [9]:
# Preprocess the dataset
test_data['task1'] = test_data['task1'].map({'non-sexist': 1, 'sexist': 0}) # Convert labels to integers
test_data['text'] = test_data['text'].astype(str) # Ensure all data are strings

In [10]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_data['text'].tolist(), train_data['task1'].tolist(), test_size=0.2)

In [11]:
# Assuming you have test_data['text'].tolist() and test_data['task2'].tolist() for test dataset
test_texts = test_data['text'].tolist()
test_labels = test_data['task1'].tolist()

In [12]:
from transformers import RobertaTokenizerFast

# Tokenize the data
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [13]:
import torch
from torch.utils.data import Dataset
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

In [15]:
from transformers import RobertaForSequenceClassification

# Define the model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predicted_labels = np.argmax(eval_pred.predictions, axis=1)
    accuracy = accuracy_score(eval_pred.label_ids, predicted_labels)
    f1 = f1_score(eval_pred.label_ids, predicted_labels, average='weighted')

    return {'accuracy': accuracy, 'f1': f1}

In [17]:
from transformers import TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
    logging_steps=10,
   load_best_model_at_end=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
)

In [18]:
from transformers import Trainer
from sklearn.metrics import accuracy_score
#compute_metrics=lambda eval_pred: {'accuracy': accuracy_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))}
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

    compute_metrics=compute_metrics
)

In [19]:
import numpy as np

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.455200,0.398345,0.829333,0.829348
2,0.290500,0.314327,0.900267,0.900246
3,0.205000,0.238631,0.953600,0.953594
4,0.076700,0.182538,0.969067,0.969066


TrainOutput(global_step=1876, training_loss=0.27880451731832584, metrics={'train_runtime': 2193.5782, 'train_samples_per_second': 13.671, 'train_steps_per_second': 0.855, 'total_flos': 6287482667733120.0, 'train_loss': 0.27880451731832584, 'epoch': 4.0})

In [20]:
# Evaluate the model on the test dataset
test_eval_result = trainer.evaluate(test_dataset)

print(f"Accuracy on test dataset: {test_eval_result['eval_accuracy']}")

Accuracy on test dataset: 0.75


In [21]:
from sklearn.metrics import f1_score

# Get predictions from the model
test_predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(test_predictions.predictions, axis=1)

# Get true labels from the TweetDataset
true_labels = test_dataset.labels

# Calculate F1 score on the test dataset
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"F1 score on test dataset: {f1}")


F1 score on test dataset: 0.7496554111983117


# **Resultados para Task1**

Datos originales, tuits en inglés,  más datos aumentados mediante técnica swapping. Sin ningún tipo de preprocesado.

#Train


```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.402700	0.373458	0.856000	0.855929
2	0.320300	0.237193	0.911467	0.911409
3	0.130400	0.133713	0.971733	0.971740
4	0.002100	0.093382	0.983467	0.983470
```

#Evaluación


```
Accuracy on test dataset: 0.7531702898550725
F1 score on test dataset: 0.7524169441792469
```



# **Datos originales, tuits en inglés, más datos aumentados mediante técnica swapping. Además se utiliza preprocesado.**

#Train


```
Epoch	Training Loss	Validation Loss	Accuracy	F1
1	0.455200	0.398345	0.829333	0.829348
2	0.290500	0.314327	0.900267	0.900246
3	0.205000	0.238631	0.953600	0.953594
4	0.076700	0.182538	0.969067	0.969066
```

#Evaluación


```
Accuracy on test dataset: 0.75
F1 score on test dataset: 0.7496554111983117
```

